<font size ="5"><b>INFO 370 Assignment 3: Webscraping</b></font>

Name: Ellen Gorbach

<font size ="3"><b>1: Legal and Ethical Issues</b></font>

1.1
Website: https://whatsinthepan.com/

1.2
Legality: What's in the Pan doesn't have a Terms of Service and the Privacy Policy is mainly about how the user's data will be protected or not rather than protection of the recipes. The robots.txt file also has no information about web scraping the recipes.

1.3
Steps to Lessen Burden:
Check terms of service (About scraping and downloading), then check robots.txt about how to scrape to make sure that you are not being a burden that cannot be supported by the website. Additionally, as the problem set instructions say, make sure to download the html as few times as possible, choosing to save the downloaded files to your machine and work from there. If that's not an option, make sure to not send too many requests to the website server in a short span of time to prevent overwhelming their system. 

<font size ="3"><b>2: Scrape the Recipes</b></font>

In [1]:
#2.1 part 1
from bs4 import BeautifulSoup
import pandas as pd
import requests
url = "https://whatsinthepan.com/recipes/"
response = requests.get(url) # retrieves the html data from the base webpage

In [2]:
#2.1 part 2
allRecipes = response.content # gets the html
allRecipesSoup = BeautifulSoup(allRecipes) # parses the html
# finds the areas that contain the links to all recipes in a category
moreCategory = allRecipesSoup.find_all("p", class_ = "more-from-category")
specializedRecipes = []
for link in moreCategory: # retrieves each of the links to the categories
    currentLink = link.find("a")
    specializedRecipes.append(currentLink.get("href"))
specializedRecipes

['https://whatsinthepan.com/category/holiday-recipes/',
 'https://whatsinthepan.com/category/maindishes/',
 'https://whatsinthepan.com/category/instant-pot/',
 'https://whatsinthepan.com/category/pasta/',
 'https://whatsinthepan.com/category/desserts/']

In [3]:
#2.1 part 3
thisRecipe = []
id = 0
for specializedRecipe in specializedRecipes:
    response = requests.get(specializedRecipe) # retrieves the html from each category page
    currentRecipes = response.content # gets the html from that category
    currentRecipesSoup = BeautifulSoup(currentRecipes) # parses the html
    # finds the areas that contain links to individual recipes
    currentRecipe = currentRecipesSoup.find_all("a", class_ = "more-link")
    for recipe in currentRecipe: # retrieves the link to each recipe
        thisRecipe.append(recipe.get("href"))
    


In [4]:
#2.1 part 4 & 2.2 & 2.3
column_names = ["id", "name", "text", "quantity", "unit", "ingredient"] # creates the variables for the dataframe
recipesdf = pd.DataFrame(columns = column_names) # creates an empty dataframe (only column names)

for currentRecipe in thisRecipe: # for each recipe
    id = id + 1 # creates an id number
    response = requests.get(currentRecipe) # retrieves the html for the recipe
    recipe = response.content # gets the html
    recipeSoup = BeautifulSoup(recipe) # parses the html
    nameStart = recipeSoup.find("div", class_="entry-content") # finds the area the name of the recipe is located
    fullName = nameStart.find("h2").text # gets the name of the recipe
    # finds where the ingredient section(s) are stored
    ingredients = recipeSoup.find_all("li", class_ = "wprm-recipe-ingredient")
    
    # Since all of the ingredients are stored the same way (aside from potential missing values which are checked for later)
    # we don't need to worry about checking for permission in terms of splitting the ingredients
    for ingredient in ingredients: 
        text = ingredient.text # extracts the name, amount, and unit of each ingredient
        # finds the amount (quantity) section of the ingredient line
        amountNode = ingredient.find("span", class_ = "wprm-recipe-ingredient-amount")
        # checks if the amount section is null and if not, extracts that value
        if amountNode is not None:
            amount = amountNode.text
        else:
            amount = None
        # finds the unit of the ingredient line
        unitNode = ingredient.find("span", class_ = "wprm-recipe-ingredient-unit")
        # checks if the unit section is null and if not, extracts that value
        if unitNode is not None:
            unit = unitNode.text
        else:
            unit = None
            # finds the name section of the ingredient line
        nameNode = ingredient.find("span", class_ = "wprm-recipe-ingredient-name")
        # checks if the name section is null and if not, extracts that value
        if nameNode is not None:
            name = nameNode.text
        else:
            name = None
        # adds all of the gathered information for each recipe/ingredient to the dataframe
        recipesdf = recipesdf.append({"id" : id, "name" : fullName, "text" : text, "quantity" : amount, "unit" : unit, "ingredient" : name}, ignore_index=True)
            
            
# https://stackoverflow.com/questions/51664292/python-attributeerror-nonetype-object-has-no-attribute-text (for the if else)
# https://www.geeksforgeeks.org/how-to-create-an-empty-dataframe-and-append-rows-columns-to-it-in-pandas/ (for the appending to df)

In [5]:
#2.3 
# preview the dataframe to make sure it makes the example in the documentation
recipesdf.head()
# I checked a few individual recipes to determine if the HTML was styled the same way for all of them
# and found that it was so I figure that as long as most of the recipes follow that same structure, 
# my code should be pretty accurate at getting the correct results back in the correct columns in my 
# dataframe. I had a little bit of difficulty with getting the loops to work the way we discussed in lecture
# so I opted for the longer but more standard way in order to get the code to work. 


,id,name,text,quantity,unit,ingredient
0,1,Boneless Turkey Breast in Garlic Herb Butter,5 tbsp butter softened (divided),5,tbsp,butter
1,1,Boneless Turkey Breast in Garlic Herb Butter,6 cloves garlic minced,6,cloves,garlic
2,1,Boneless Turkey Breast in Garlic Herb Butter,"2 tbsp fresh herbs (thyme, rosemary, sage and ...",2,tbsp,"fresh herbs (thyme, rosemary, sage and parsley)"
3,1,Boneless Turkey Breast in Garlic Herb Butter,1 tsp fresh lemon zest,1,tsp,fresh lemon zest
4,1,Boneless Turkey Breast in Garlic Herb Butter,1/2 tsp salt,1/2,tsp,salt


In [6]:
#2.4
# STORE DATAFRAME IN A FILE
recipesdf.to_csv('recipesdf.csv')

# Taking a look at some of my values in the csv file, I noticed that some of the quantities are framed as dates
# because fractions in Excel turn into dates (Ex: 1/2 turned into Jan-2) which doesn't seem to have an impact on my
# analysis below but wanted to give a heads up. 

<font size ="3"><b>3: Sweet or Salty Foods</b></font>

In [9]:
#3.1
recipesDF = pd.read_csv("recipesdf.csv")
recipesDF["saltExists"] = recipesDF["ingredient"].str.contains("salt", regex=False)
salt = recipesDF.groupby("id")["saltExists"].any().sum()
print(salt)

recipesDF["sugarExists"] = recipesDF["ingredient"].str.contains("sugar", regex=False)
sugar = recipesDF.groupby("id")["sugarExists"].any().sum()
print(sugar)

# There are more recipes that use salt than use sugar at any point in the process

39
14


In [8]:
#3.2 
# COUNT VALUES OF SALT AND SUGAR AND DETERMINE WHICH THERE IS MORE OF
#from fractions import Fraction

#recipesdf[""] = recipesdf["quantity"].Fraction(recipdesdf["quantity"])
#recipesdf["result"] = recipesdf["quantity"].str.split("/").apply(lambda x: float(x[0]) / float(x[1]))
#saltQuantity = recipesdf.groupby("saltExists")["quantity"].sum()
#saltQuantity
#recipesdf

# Since there are certain quantities which are fractions, I would need to create a function that splits the 
# fractions into their float counterparts and after many attempts based on internet searches (see some above)
# I haven't been able to succesfully achieve that transformation. Because I can't transform those into floats,
# I can't do the sum of all quantities of sugar and sum of all quantities of salt and compare the two.
# Additionally, the it's really difficult to compare quantities when the units are different. Sometimes you need
# a pinch of salt (and who can say how much that is compared to a teaspoon) which means that even if I got the 
# splitting sorted out, I would still not be gathering accurate information about absolute quantities.

I spent around 8 hours on this assignment